In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import torch
import torch.nn as nn
import torch.optim as optim
import ot
import sys
import os
import tarfile
import scipy.io
import h5py
import gzip
from scipy.sparse import issparse
from scipy.sparse import csr_matrix

In [28]:
adata = sc.read_10x_h5(filename="/workspace/ImputationOT/data/matrix.h5")
print(adata)

/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 8371 × 18082
    var: 'gene_ids', 'feature_types', 'genome'


/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [4]:
adata = sc.read_h5ad("/workspace/ImputationOT/data/failing.h5ad")
print(adata)

AnnData object with n_obs × n_vars = 269794 × 45068
    obs: 'orig.ident', 'tech', 'condition', 'Sex', 'id', 'Names', 'CondTech'
    var: 'name'
    obsm: 'X_umap'


In [12]:
print(adata.obs['orig.ident'])

H_ZC-11-292_TAAGTGCAGCAGGTCA    H_ZC-11-292
H_ZC-11-292_ACAGCCGGTCATACTG    H_ZC-11-292
H_ZC-11-292_AACTCCCTCTTTAGTC    H_ZC-11-292
H_ZC-11-292_CTGATCCGTTATTCTC    H_ZC-11-292
H_ZC-11-292_GCACTCTTCTCGTATT    H_ZC-11-292
                                   ...     
HDCM8_TTAGGCAGTTTGTGTG-1              HDCM8
HDCM8_TTCCCAGAGAGGTAGA-1              HDCM8
HDCM8_TTTCCTCCAGATGGCA-1              HDCM8
HDCM8_TTTGTCATCGCGATCG-1              HDCM8
HDCM8_GCACTCTCATCGGACC-1              HDCM8
Name: orig.ident, Length: 269794, dtype: category
Categories (45, object): ['HDCM1', 'HDCM3', 'HDCM4', 'HDCM5', ..., 'TWCM-13-285', 'TWCM-14-173', 'TWCM-LVAD2', 'TWCM-LVAD3']


In [37]:
adata = sc.read_h5ad("/workspace/ImputationOT/data/healthy.h5ad")
print(adata)

AnnData object with n_obs × n_vars = 486134 × 33538
    obs: 'celltype', 'sampleID', 'SampleType', 'Source', 'DonorID', 'Organ', 'Tissue', 'TissueDetail', 'Gender', 'Age', 'AgeBin', 'DeathType', 'protocol', 'Race'
    var: 'gene_ids'
    uns: 'celltype_colors'
    obsm: 'X_umap'


In [ ]:
print(adata.obs[''])

In [29]:
X = adata.X
print("Matrix Shape:", X.shape)
print("Density:", np.count_nonzero(X) / (X.shape[0] * X.shape[1]))
# print("Density:", X.nnz / (X.shape[0] * X.shape[1]))
print("Minimum Value:", X.min())
print("Maximum Value:", X.max())

Matrix Shape: (9409, 60970)
Density: 0.04190173273600859
Minimum Value: 0.0
Maximum Value: 13.540438


In [31]:
print(adata.obs['Batch'])

index
Cell9517     Batch1
Cell5323     Batch1
Cell6864     Batch1
Cell9648     Batch1
Cell1185     Batch1
              ...  
Cell55408    Batch6
Cell52882    Batch6
Cell51973    Batch6
Cell57940    Batch6
Cell53110    Batch6
Name: Batch, Length: 12097, dtype: category
Categories (6, object): ['Batch1', 'Batch2', 'Batch3', 'Batch4', 'Batch5', 'Batch6']
